In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
%load_ext autoreload
%autoreload 

In [ ]:
labels = pd.read_csv('./data/hasyv2/hasy-data-labels.csv')

In [ ]:
label_dict = {"times": 514, "plus": 196, "minus": 195, 'div': 526, 'eq': 621}
rel_list = [514, 196, 195, 526, 621]

In [ ]:
rel_labels = labels[labels.symbol_id.isin(rel_list)]
rel_labels.head(3)

In [ ]:
ROOTDIR = "./data/hasyv2/"

img = np.asarray(Image.open(ROOTDIR + "hasy-data/v2-05568.png"))

plt.imshow(img)
img.shape

In [ ]:
# Create datafolder
import os
import shutil
from pathlib import Path
HASY_DIR = "./data/hasyv2/"
OPERATOR_DIR = "./data/all_data/mul"
rename_map = {514: "mul", 196: "plus", 195: "minus", 526: "div", 621: "eq"}

if not os.path.exists(OPERATOR_DIR):
    os.makedirs(OPERATOR_DIR)

minus = 0
plus = 0
mul = 0
div = 0
eq = 0
 

current_label = 0
for index, row in rel_labels.iterrows():
    if current_label != row.symbol_id: 
        current_label = row.symbol_id
    if rename_map[row.symbol_id] == "mul":
        mul += 1
        filename = "{}_{}".format(rename_map[row.symbol_id],mul)
        old_path = HASY_DIR + row.path
        new_path = "{}/{}.png".format(OPERATOR_DIR,filename)
        shutil.copy(old_path, new_path)

    """
    elif rename_map[row.symbol_id] == "minus":
        minus += 1
        filename = "{}_{}".format(rename_map[row.symbol_id],minus)
    elif rename_map[row.symbol_id] == "plys":
        mul += 1
        filename = "{}_{}".format(rename_map[row.symbol_id],plus)
    elif rename_map[row.symbol_id] == "div":
        div += 1
        filename = "{}_{}".format(rename_map[row.symbol_id],div)
    elif rename_map[row.symbol_id] == "eq":
        eq += 1
        filename = "{}_{}".format(rename_map[row.symbol_id],eq)
    """


    

In [3]:
import os
def update_datasheet(image_dir = "./data/operators"):
    """ 
    Updates datasheet to fit current content of imagefolder 
    Filename must include correct label [plus, minus, mul, div, eq]
    """
    print("Updating Datasheet")
    data = pd.DataFrame(columns = ["path", "label"])

    i = 0
    for file in os.listdir(image_dir):
        if "zero" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 0}, index = [0])

        elif "one" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 1}, index = [0])

        elif "two" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 2}, index = [0])

        elif "three" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 3}, index = [0])

        elif "four" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 4}, index = [0])
        
        elif "five" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 5}, index = [0])
            
        elif "six" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 6}, index = [0])
       
        elif "seven" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 7}, index = [0]) 
                
        elif "eigth" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 8}, index = [0])

        elif "nine" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 9}, index = [0])
        
        elif "plus" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 10}, index = [0])
            
        elif "minus" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 11}, index = [0])
       
        elif "mul" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 12}, index = [0])
        
        elif "div" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 13}, index = [0])
       
        elif "eq" in file:
            newrow = pd.DataFrame(columns = ["path", "label"],data = {"path": file, "label": 14}, index = [0])
        i += 1
        
        data = data.append(newrow, ignore_index = True)
        data = data.reset_index(drop=True)
    data.to_csv(image_dir + "/datasheet.csv")

In [39]:
#from importlib import reload
#import utils
#reload(utils)
#from utils import update_datasheet
update_datasheet(image_dir = "./data/train_dil")

Updating Datasheet


In [13]:
""" Reshape to 45x45 """
basepath = "./data/operators_test/"
!rm ./data/operators_test/.DS_Store
for file in os.listdir(basepath):
    image = Image.open(basepath+file)
    image = image.resize((45,45), Image.ANTIALIAS)
    image.save(basepath+file)


rm: ./data/operators_test/.DS_Store: No such file or directory


In [4]:
import torch
from skimage.transform import resize
from skimage.color import rgb2gray

def get_operators_from_original(filepath = "./data/original_operators.png"):
    """
    Takes in filepath the original image with all operators
    Returns each operator in 32x32 and greyscale
    """
    operators = Image.open(filepath)
    img = np.asarray(operators)
    img_grey = rgb2gray(img)
    
    N = 316
    plus = img[:, 0:N]
    equals = img[:, N:2*N]
    min_offset = 75
    minus = img[:, 2*N + min_offset :3*N + min_offset]
    div_offset = 125
    div = img[:, 3*N + div_offset:4*N + div_offset]
    mul_offset = 155
    mul = img[:, 4*N + mul_offset:-1]
    
    operators = [plus, minus, div, mul, equals]
    operators = [resize(operator, (45, 45)) for operator in operators]
    operators = [rgb2gray(operator) for operator in operators]
    op_tensors = [torch.from_numpy(operators[i].astype(np.float32)) for i in range(len(operators))]

    return operators

In [5]:
def add_rotated_operators(output_path='/data/train/'):
    """
    Retrieves the original operators, rotates and puts back in train-folder
    """
    operators = get_operators_from_original()
    name_map = ["plus", "minus", "div", "mul", "equals"]
    
    for operator, name in zip(operators, name_map):
        image = Image.fromarray(np.uint8(cm.Greys(operator)*255))

        for deg in range(15,360,15):
            rotated = image.rotate(deg, PIL.Image.NEAREST, expand = 1, fillcolor = (0, 0, 0))
            rotated = rotated.resize((45,45), Image.ANTIALIAS)
            rotated.save("{}{}_rot_{}.png".format(output_path, name, deg))
            
add_rotated_operators(output_path="./data/orig_rot/")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import PIL

basepath = "./data/operators_test/"
original_images = ["plus", "minus", "div", "eq", "mul"]
 
# Create an Image object from an Image
for img in original_images:
    
    colorImage  = Image.open("{}{}.png".format(basepath,img))
 
    for deg in range(21, 360, 21):

        rotated = colorImage.rotate(deg, PIL.Image.NEAREST, expand = 1, fillcolor = (255, 255, 255))
        rotated = rotated.resize((45,45), Image.ANTIALIAS)
        rotated.save("{}{}_rot_{}.png".format("./data/operators_train/",img, deg))


In [ ]:
!rm ./data/all_data/plus/.DS_Store

In [12]:

def add_base_training(output_path, basepath = "./data/all_data/", max_img_per_class = 1000, square_size = 5):
    """
    Collects data from the basic-dataset
    """
    folders = ["div", "minus", "plus", "eq", "mul", "zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]

    for folder in folders:
        i = 500
        folderpath = basepath+folder+"/"
        for img in os.listdir(folderpath):
            image = Image.open(folderpath+img)
            image = np.asarray(image.resize((45,45), Image.ANTIALIAS))
            image = rgb2gray(image)
            image = erosion(image, square(square_size))
            image = Image.fromarray(np.uint8(cm.Greys(image)*255))
            image.save("{}{}_{}.png".format(output_path, folder, i))
            i-=-1
            if i > 500 + max_img_per_class:
                break

In [10]:
def threshold(img):
    img[img > img.mean()] = 255
    img[img <= img.mean()] = 0
    return img
    

def add_candidates(output_path, basepath= "./data/candidates/"):
    
    for img_name in os.listdir(basepath):
        image = Image.open(basepath + img_name)
        image = np.asarray(image.resize((45,45), Image.ANTIALIAS))
        image = rgb2gray(image)
        image = threshold(image)
        image = Image.fromarray(np.uint8(cm.Greys(image)*255))
        
        for deg in range(21, 360, 21):
            rotated = image.rotate(deg, PIL.Image.NEAREST, expand = 1, fillcolor = 0)
            rotated = rotated.resize((45,45), Image.ANTIALIAS)
            rotated.save("{}{}_rot_{}".format(output_path, deg, img_name))

#add_candidates("./data/test/")


In [14]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage.morphology import erosion, dilation, square
from skimage.color import rgb2gray
from PIL import Image
import PIL

def create_training_set(train_dir):
    """
    Pipeline-function that adds images to folder
    """
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    
    #add_base_training(train_dir, max_img_per_class= 300)
    #add_rotated_operators(train_dir)
    #add_candidates(train_dir)
    update_datasheet(train_dir)

create_training_set("./data/train_candidates/")

Updating Datasheet
